In [1]:
import json

In [2]:
big = "memoryalpha"
small = "stexpanded"
top_k = 1

gold_pairs_file = "./data/gold_pairs/" + big + "-" + small + "_formatted.txt"
matched_communities_file = "./results/matched_communities_leiden/top_10_pairs/" + small + "-" + big + "_top_10_pairs.json"

communities_big_file = "./results/communities_leiden/" + big + "/final.txt"
communities_small_file = "./results/communities_leiden/" + small + "/final.txt"

small_communities = []
big_communities = []
gold_pairs = []
matched_communities = []

In [3]:
with (open(communities_big_file) as cbf, open(communities_small_file) as csf,
      open(gold_pairs_file) as gpf, open(matched_communities_file) as mcf):
    
    for line in csf:
        numbers_set = {int(num) for num in line.strip().split(" ")}
        small_communities.append(numbers_set)

    for line in cbf:
        numbers_set = {int(num) for num in line.strip().split(" ")}
        big_communities.append(numbers_set)
    
    for line in gpf:
        numbers_list = [int(num) for num in line.strip().split(";")]
        gold_pairs.append(numbers_list)

    matched_communities_load = json.load(mcf)
    matched_communities = [[int(key), [int(values[i][0]) for i in range(0, top_k)]] for key, values in matched_communities_load.items()]

In [4]:
matched_communities

[[0, [0]],
 [1, [379]],
 [2, [856]],
 [3, [460]],
 [4, [913]],
 [5, [3]],
 [6, [912]],
 [7, [913]],
 [8, [911]],
 [9, [912]],
 [10, [648]],
 [11, [97]],
 [12, [2101]],
 [13, [97]],
 [14, [1843]],
 [15, [1665]],
 [16, [2234]],
 [17, [854]],
 [18, [935]],
 [19, [233]],
 [20, [2139]],
 [21, [522]],
 [22, [330]],
 [23, [536]],
 [24, [1901]],
 [25, [413]],
 [26, [1902]],
 [27, [623]],
 [28, [1]],
 [29, [99]],
 [30, [97]],
 [31, [97]],
 [32, [452]],
 [33, [648]],
 [34, [1753]],
 [35, [648]],
 [36, [913]],
 [37, [648]],
 [38, [1661]],
 [39, [1665]],
 [40, [102]],
 [41, [1277]],
 [42, [539]],
 [43, [1037]],
 [44, [1153]],
 [45, [913]],
 [46, [409]],
 [47, [348]],
 [48, [1037]],
 [49, [97]],
 [50, [1037]],
 [51, [1635]],
 [52, [603]],
 [53, [70]],
 [54, [648]],
 [55, [2234]],
 [56, [1665]],
 [57, [85]],
 [58, [408]],
 [59, [1658]],
 [60, [842]],
 [61, [1037]],
 [62, [434]],
 [63, [97]],
 [64, [1022]],
 [65, [1638]],
 [66, [211]],
 [67, [123]],
 [68, [160]],
 [69, [917]],
 [70, [468]],
 [71, [10

In [5]:
found = 0
all_pairs = len(gold_pairs)

for gold_pair in gold_pairs:
    for matched_community in matched_communities:
        community_1 = small_communities[matched_community[0]]
        community_2 = set()
        for i in range(0, top_k):
            community_2.update(big_communities[matched_community[1][i]])
            if gold_pair[0] in community_1 and gold_pair[1] in community_2:
                found += 1
                print(matched_community[0], matched_community[1], gold_pair)
                break
            
print("Gold pairs in matched communities top " + str(top_k) + ": " + str(found / all_pairs * 100) + "%")

590 [97] [9762, 2178]
Gold pairs in matched communities top 1: 0.056211354693648116%


In [6]:
found_big = 0
found_small = 0
for gold_pair in gold_pairs:
    for big_community in big_communities:
        if gold_pair[0] in big_community:
            found_big += 1
            break
    for small_community in small_communities:
        if gold_pair[1] in small_community:
            found_small += 1
            break

print("Big: " + str(found_big / all_pairs * 100) + "%")
print("Small: " + str(found_small / all_pairs * 100) + "%")

Big: 100.0%
Small: 100.0%
